# Submission Information:

### Team Member 1:
* UNI:  asp2197
* Name: Abhay Pawar

### Team Member 2 [optional]:
* UNI:  vb2424
* Name: Vijay Balaji

# Step0 - Import Libraries, Load Data [0 points]

This is the basic step where you can load the data and create train and test sets for internal validation as per your convinience.

In [6]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import feature_selection
from sklearn import preprocessing

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_regression,RFE,RFECV
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [7]:
data = pd.read_csv("data/data.csv")
holdout = pd.read_csv("data/holdout.csv")
data.head()

sub=data.subscribed.copy().as_matrix()
data.drop(['subscribed'],axis=1,inplace=True)
hold_ids=holdout.ID.copy()
holdout.drop(['ID'],axis=1,inplace=True)
y=np.zeros(len(sub))
for i in range(len(sub)):
    if sub[i]=='no':
        y[i]=0
    elif sub[i]=='yes':
        y[i]=1
        
data.drop(['duration'],axis=1,inplace=True)
holdout.drop(['duration'],axis=1,inplace=True)

# Step1 - Exploration and Preparation [10 points]

In this step, we expect you to look into the data and try to understand it before modeling. This understanding may lead to some basic data preparation steps which are common across the two model sets required.

We did initial exploration of the data in excel using pivot tables and filters as it is much more convenient. We realized that the train and holdout data don't have the same categories for some categorical variables. We compared the categories in these variables in train and holdout data and did grouping so that both datasets have same categories. The grouping was done by looking at every categorical feature individually and grouping catgories which have very similar event rates. Following code does the grouping.
We also checked if there are any outliers. There are no outliers in the continuous features. prev_days had a value 999 which means that customer wasn't contacted. We have converted this feature into a binary and tells if the customer was contacted or not

In [8]:
def loangroup(dataset):
    temp = dataset['loan'].copy()
    temp[dataset['loan']=='unknown'] = 'no'
    dataset['loan'] = temp
def housinggroup(dataset):
    temp = dataset['housing'].copy()
    temp[dataset['housing']=='unknown'] = 'yes'
    dataset['housing'] = temp
def credit_defaultgroup(dataset):
    temp = dataset['credit_default']
    temp[dataset['credit_default']=='unknown'] = 'yes'
    dataset['credit_default'] = temp
def educationgroup(dataset):
    temp = dataset['education'].copy()
    temp[dataset['education']=='unknown'] = 'illiterate'
    dataset['education'] = temp
def jobgroup(dataset):
    temp = dataset['job'].copy()
    temp[dataset['job']=='unknown'] = 'technician'
    temp[dataset['job']=='housemaid'] = 'technician'
    temp[dataset['job']=='entrepreneur'] = 'services'
    dataset['job'] = temp
def maritalgroup(dataset):
    temp = dataset['marital_status'].copy()
    temp[dataset['marital_status']=='unknown'] = 'single'
    dataset['marital_status'] = temp

def campaigngroup(dataset):
    temp = dataset['campaign'].copy()
    temp[dataset['campaign']<5] = 1
    temp[(dataset['campaign']<8) & (dataset['campaign']>=5)] = 2
    temp[dataset['campaign']>=8] = 3
    dataset['campaign'] = temp
    dataset['campaign'] = dataset['campaign'].astype('category')
    
def monthgroup(dataset):
    temp = dataset['month'].copy()
    temp[dataset['month']=='nov'] = 'jun'
    temp[dataset['month']=='aug'] = 'jun'
    dataset['month'] = temp

def prev_daysgroup(dataset):
    temp = dataset['prev_days'].copy()
    temp.iloc[np.where(dataset['prev_days']!=999)] = 0
    temp.iloc[np.where(dataset['prev_days']==999)] = 1
    dataset['prev_days_binary'] = temp    

def agegroup(dataset):
    temp=dataset['age'].copy()
    temp[dataset['age']<=20] = 20
    temp[(dataset['age']>20) & (dataset['age']<=26)] = 26
    temp[(dataset['age']>26) & (dataset['age']<=30)] = 30
    temp[(dataset['age']>30) & (dataset['age']<=38)] = 38
    temp[(dataset['age']>38) & (dataset['age']<=51)] = 51
    temp[(dataset['age']>51) & (dataset['age']<=61)] = 61
    temp[dataset['age']>61] = 62
    dataset['age_group']=temp

In [9]:
loangroup(data)
loangroup(holdout)
housinggroup(data)
housinggroup(holdout)
credit_defaultgroup(data)
credit_defaultgroup(holdout)
educationgroup(data)
educationgroup(holdout)
jobgroup(data)
jobgroup(holdout)
maritalgroup(data)
maritalgroup(holdout)
campaigngroup(data)
campaigngroup(holdout)
monthgroup(data)
monthgroup(holdout)
prev_daysgroup(data)
prev_daysgroup(holdout)
agegroup(data)
agegroup(holdout)

G:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
cat = ['age_group','job','marital_status','education','prev_outcomes','month','contact','campaign','day_of_week']
for c in cat:
    data[c] = data[c].astype('category')
    holdout[c] = holdout[c].astype('category')
X = pd.get_dummies(data)
X_holdout = pd.get_dummies(holdout)
X=X.drop(X.columns[[29, 31,33,35]],axis=1)
X_holdout=X_holdout.drop(X_holdout.columns[[29, 31,33,35]],axis=1)


# Step2 - ModelSet1 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set1:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. Any classification algorithm covered in class apart from tree-based models can be tested here.

Feature Engineering/Selection: 

We tried to create a new feature using prev_days and prev_outcome. Reason being that customers with prev_days==999 have prev_outcome as unknown and hence, there is high correlation. But, this feature did not add any improvement in logistic regression and hence, we decided to not use it. We converted prev_days into a binary variable for logistic regression as 999 value is too high. We used prev_days as it is for tree based models as 999 value wouldn't affect the results.

We tried using the F scores from f_classif and RFECV to select the features. RFECV gave better results and hence, we have used it for all the models.

Validation:
Since, cross-val-score computes scores on random splits on data, we initially created a single 20% test data to check the performance of models. But, improvement in AUC on this test data didn't necessarily convert into improvement on the holdout on Leaderboard. So, we shifted to using cross_val_score. 

Classifiers used:
We used logistic regression, KNN, Naive bayes in this stage.



In [11]:
#F scores
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=11)
F,p=feature_selection.f_classif(X_train.as_matrix(),y_train)
Fscore=pd.DataFrame(X_train.columns)
Fscore['F_value']=F
Fscore['p_value']=p
#Fscore.to_csv('Dummies_fscore.csv')

For logistic regression we first selected the features using RFECV and then used gridsearch to find optimal C with l2 penalty

In [12]:
#RFECV on logistic regression
estimator = LogisticRegression()
selector = RFECV(estimator, step=1, cv=2,scoring='roc_auc')
selector = selector.fit(X_train, y_train)
print selector.support_ 
print selector.ranking_

[False False  True False False False  True False False False  True False
  True False False False False False False False False False False False
 False False False False  True False False False  True  True  True False
  True  True  True False False  True False False False  True False  True
  True  True  True  True False False  True  True  True False]
[33 36  1 24 32 25  1 34 38 11  1 18  1 15 22  5 29 30 27 19 12 28  9  4  3
 31  8 20  1 21 37  7  1  1  1  6  1  1  1 35 26  1 16 17 14  1 13  1  1  1
  1  1 23  2  1  1  1 10]


In [13]:
print selector.support_
k=[]
for i in range(len(selector.support_)):
    if selector.support_[i]==False:
        k.append(i)
#k contains the columns to be dropped

[False False  True False False False  True False False False  True False
  True False False False False False False False False False False False
 False False False False  True False False False  True  True  True False
  True  True  True False False  True False False False  True False  True
  True  True  True  True False False  True  True  True False]


In [15]:
param_grid={'logisticregression__C':np.arange(0.001,0.02,0.002)}
gd=GridSearchCV(LogisticRegression(penalty='l2',param_grid=param_grid,scoring='roc_auc',cv=3)
gd.fit(X.drop(data.columns[[k]],axis=1),y)
gd.best_params_ 
#lr=LogisticRegression()
#lr.fit(data.drop(data.columns[[k]],axis=1),y)
#plr=make_pipeline(StandardScaler(),lr)
#plr.fit(data_train.drop(data_train.columns[[k]],axis=1),y_train)
#lr.fit(data_train,y_train)
#y_predict_lr = lr.predict_proba(data_test)
#print roc_auc_score(y_test,y_predict_lr[:,1])
#scores=cross_val_score(lr,data.drop(data.columns[[k]],axis=1),y,cv=5,scoring='roc_auc')
#scores
#scores.mean()


SyntaxError: invalid syntax (<ipython-input-15-eaa6f571d2e7>, line 3)

In [ ]:
lr=make_pipeline(StandardScaler(),LogisticRegression(penalty='l2',C=0.005))
scores=cross_val_score(lr,data.drop(data.columns[k],axis=1),y,cv=5,scoring='roc_auc').mean()

In [ ]:
#adaboost:
sgd=SGDClassifier(loss='log', penalty='l2', alpha=1, l1_ratio=0.5, fit_intercept=True, n_iter=5, shuffle=True, verbose=0, 
              epsilon=0.1, n_jobs=1, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, class_weight=None, 
              warm_start=False, average=False)
abc=AdaBoostClassifier(base_estimator=SVC(), n_estimators=50, learning_rate=1.0, algorithm='SAMME', random_state=None)

abc.fit(X_train,y_train)
y_predict=abc.predict_proba(X_test)
print roc_auc_score(y_test,y_predict[:,1])


For KNN, we tried with scaling the features. But, it gave worst results as compared to without scaling. Probable reason could be that high predictive features which should have higher weight get scaled down and hence, contribute as much as non-predictive features. Hence, worsening the performance. We also tried dropping low F score features, but it didn't give much improvement.

In [224]:

knn=KNeighborsClassifier(n_neighbors=40, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', 
                     metric_params=None, n_jobs=1)
knn_pipe=make_pipeline(knn)
knn_pipe.fit(X_train,y_train)
y_predict_knn=knn.predict_proba(X_test)[:,1]
y_predict_knn_train=knn.predict_proba(X_train)[:,1]

print roc_auc_score(y_test,y_predict_knn)
print roc_auc_score(y_train,y_predict_knn_train)


0.761687824985
0.831405324396


In [168]:
#To run on holdout
knn=KNeighborsClassifier(n_neighbors=25, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', 
                     metric_params=None, n_jobs=1)
knn_pipe=make_pipeline(knn)
knn_pipe.fit(X_train,y_train)
y_predict_knn=knn_pipe.predict_proba(X_holdout_drop)[:,1]
y_predict_knn_train=knn_pipe.predict_proba(X_test)[:,1]

print roc_auc_score(y_test,y_predict_knn_train)

0.757314447918


Naive Bayes: We used the gaussian assumption which won't be true for all the features. Still NB performs decent. We tried dropping features which are highly correlation. But, this did not much improvement. Only dropping one feature gave better results

In [225]:
#Drop on basis of F score: [41,15,10,31,32,45,28, ] var removed due to feature engineering: 1, 51, 55,2, 49
#Drop due to correlation: 3,6
#Due to binary: 28, 30,32,17,34

#second stage low power: 29,53,42,16,43,25,24,23,17,12,44,20   18, 38, 4
#Corr second stage: 46, 38, 
k=[46]
X_drop=X.drop(X.columns[k],axis=1)
X_holdout_drop=X_holdout.drop(X_holdout.columns[k],axis=1)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_drop,y,random_state=11)

In [226]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB
nbg=GaussianNB()
#BernoulliNB
#GaussianNB
nbg.fit(X_train,y_train)
y_predict_nbg=nbg.predict_proba(X_test)[:,1]
y_predict_nbg_train=nbg.predict_proba(X_train)[:,1]
print roc_auc_score(y_test,y_predict_nbg)
print roc_auc_score(y_train,y_predict_nbg_train)

0.766064318262
0.769425660311


Ensembles: We tried three different strategies to create ensemble model. 
1. Poor man's stacking: Building a model over the probabilities given by individual models
2. Weighted averaging of probabilities from different model
3. Weigthed average of rank of each customer from different models. Since, AUC depends only on how the customers are ranked by the probabilites, averaging the rank from probabilites from different models makes sense.
2 and 3 gave improvement. 1 did not give much improvement

In [244]:
#1. Poor man's stacking. Used GBC and logistic. Both, didn't give improvement.
#X_train_ens=pd.DataFrame()
X_train_ens=pd.DataFrame(y_pred_train_plr)
X_train_ens.columns=['y_pred_train_plr']
X_train_ens['y_predict_knn_train']=y_predict_knn_train
X_train_ens['y_predict_nbg_train']=y_predict_nbg_train
X_train_ens['y_pred_train_gbc']=y_pred_train_gbc
X_train_ens.head()
X_test_ens=pd.DataFrame(y_predict_plr)
X_test_ens.columns=['y_predict_plr']
X_test_ens['y_predict_knn_train']=y_predict_knn
X_test_ens['y_predict_nbg_train']=y_predict_nbg
X_test_ens['y_pred_train_gbc']=y_predict_gbc
gbc1=GradientBoostingClassifier(loss='exponential', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', 
                           min_samples_split=2, min_samples_leaf=2, min_weight_fraction_leaf=0.0, max_depth=10, 
                           min_impurity_split=1e-07, init=None, random_state=11, max_features=None, verbose=0, 
                           max_leaf_nodes=20, warm_start=False, presort='auto')
gbc1.fit(X_train_ens,y_train)
y_predict_ens=gbc1.predict_proba(X_test_ens)[:,1]
y_predict_ens_train=gbc1.predict_proba(X_train_ens)[:,1]

print roc_auc_score(y_test,y_predict_ens)
print roc_auc_score(y_train,y_predict_ens_train)

0.768728306176
0.878260966781


In [249]:
#Weighted average of probabilites
auc_max=0.5
for i in range(0,6):
    for j in range(0,6):
        for k in range(0,6):
            if i+k+j!=0:
                y_ens=ensembles([y_predict_plr,y_predict_nbg,y_predict_gbc],[i+0.0,j+0.0,k+0.0])
                auc=roc_auc_score(y_test,y_ens)
                if auc>auc_max:
                    auc_max=auc
                    maxi=i
                    maxj=j
                    maxk=k
print auc_max,maxi,maxj,maxk

#auc_max,maxi,maxj 0.7952718796 2 3
#0.795813061279 4 5 1 with knn

0.794342804087 2 0 5


In [16]:
#Rank ensembles
def rank_pred(y):
    y_pred=pd.DataFrame(y)
    y_pred.columns=['y_pred']
    y_pred_sorted=y_pred.sort(columns='y_pred').reset_index()
    y_pred_sorted['rank']=pd.Series(np.zeros(len(y_pred)))
    for i in range(len(y_pred_sorted)):
        y_pred_sorted.set_value(i, 'rank', (i+0.0)/len(y_pred_sorted), takeable=False)
    y_pred_fin=y_pred_sorted.sort(columns='index').reset_index(drop=True)
    y_pred_rank=y_pred_fin['rank']
    return y_pred_rank
    #y_pred_rank


# Step3 - ModelSet2 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set2:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. We encourage you to try decition tree, random forest and gradient boosted tree methods here and pick the one which you think works best.

Models used:

We used random forest and gradient boosted trees in this stage.

Random Forest

In [ ]:
rf=RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, min_samples_split=10, min_samples_leaf=10, 
                       min_weight_fraction_leaf=0.0,max_features='auto', max_leaf_nodes=None, min_impurity_split=1e-07, 
                       bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, 
                       class_weight=None)
#cross_val_score()
#param_grid={'min_samples_split':np.linspace(1,1000,20)}
#scores=cross_val_score(rf,data,y,cv=5,scoring='roc_auc')
#print scores
#print scores.mean()
# try on whole data first
rf.fit(data_train,y_train)
y_predict_rf = rf.predict_proba(data_test)

In [ ]:
selector = RFECV(rf, step=1, cv=2,scoring='roc_auc')
selector = selector.fit(data_train, y_train)
print selector.support_ 
print selector.ranking_

In [ ]:
k=[]
for i in range(len(selector.support_)):
    if selector.support_[i]==False:
        k.append(i)

In [ ]:
scores=cross_val_score(rf,data.drop(data.columns[[k]],axis=1),y,cv=5,scoring='roc_auc')
print scores
print scores.mean()

In [ ]:
rf.fit(data_train.drop(data_train.columns[[k]],axis=1),y_train)
y_res_rf_train=rf.predict_proba(data_test.drop(data_test.columns[[k]],axis=1))

In [ ]:
rf.fit(data,y)
y_res_rf=rf.predict_proba(holdout)

output=pd.DataFrame(hold_ids)
#output['ID']=hold_ids
output['subscribed']=y_pred[:,1]
output.to_csv("rf.csv",index=False)

Gradient Boosting

In [ ]:
gbc2=GradientBoostingClassifier(min_samples_leaf=20,min_samples_split=2,max_depth=3)
#param_grid={'max_leaf_nodes':[None,5,10,15,25,50]}
#gd=GridSearchCV(gbc2,param_grid=param_grid,scoring='roc_auc',cv=3)
#gd.fit(data_train,y_train)
#gd.best_params_ 
#scores=cross_val_score(gbc2,data.drop(data.columns[[k]],axis=1),y,cv=5,scoring='roc_auc')
#scores
#scores.mean()
gbc2.fit(data_train,y_train)
y_predict_gbc2 = gbc2.predict_proba(data_test)
#roc_auc_score(y_test,y_predict_gbc2[:,1])

In [ ]:
selector = RFECV(gbc2, step=1, cv=2,scoring='roc_auc')
selector = selector.fit(data_train, y_train)
print selector.support_ 
print selector.ranking_

In [ ]:
print selector.support_
kgbc=[]
for i in range(len(selector.support_)):
    if selector.support_[i]==False:
        kgbc.append(i)
kgbc

In [ ]:
gbc2.fit(data_train.drop(data_train.columns[[kgbc]],axis=1),y_train)
y_res_gbc_train=gbc2.predict_proba(data_test.drop(data_test.columns[[kgbc]],axis=1))

In [ ]:
gbc2.fit(data,y)
y_res_gbc=gbc2.predict_proba(holdout)

output=pd.DataFrame(hold_ids)
#output['ID']=hold_ids
output['subscribed']=y_pred[:,1]
output.to_csv("gbcdefault.csv",index=False)

# Step4 - Ensemble [20 points + 10 Bonus points]

In this step, we expect you to use the models created before and create new predictions. You should definitely try poor man's stacking but we encourage you to think of different ensemble techniques as well. We will judge your creativity and improvement in model performance using ensemble models and you can potentially earn 10 bonus points here.

In [5]:
#write code below, you can make multiple cells
#assert True

In [ ]:
def weightedprobs(w1,w2,w3):
    y_predict_ensemble =((w1/(w1+w2+w3) * y_predict_lr[:,1])  + (w2/(w1+w2+w3) * y_predict_gbc2[:,1]) + (w3/(w1+w2+w3) * y_predict_rf[:,1]))
    return y_predict_ensemble

In [ ]:
maxi=0
maxj=0
maxk=0
maxl=0
max=0
for i in [0,0.1,0.2,0.3,0.4,0.5]:
    for j in [0,0.1,0.2,0.3,0.4,0.5]:
        for k in [0,0.1,0.2,0.3,0.4,0.5]:
                if i + j + k!=0:
                    y_predict_ensemble = weightedprobs(i,j,k)
                    auc = roc_auc_score(y_test,y_predict_ensemble)
                    if auc>max:
                        maxi = i
                        maxj = j
                        maxk = k
                        max = auc

In [ ]:
from sklearn.ensemble import VotingClassifier

eclf = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('gbc', gbc)], voting='soft',weights=[maxi,maxk,maxj])
eclf.fit(data_train,y_train)
y_predict_ensemble = eclf.predict_proba(data_test)
roc_auc_score(y_test,y_predict_ensemble[:,1])

In [ ]:
eclf.fit(data,y)
y_pred=eclf.predict_proba(holdout)

output=pd.DataFrame(hold_ids)
#output['ID']=hold_ids
output['subscribed']=y_pred[:,1]
output.to_csv("ensemblefulldata.csv",index=False)

In [ ]:
def rank_pred(y):
    y_pred=pd.DataFrame(y)
    y_pred.columns=['y_pred']
    y_pred_sorted=y_pred.sort(columns='y_pred').reset_index()
    y_pred_sorted['rank']=pd.Series(np.zeros(len(y_pred)))
    for i in range(len(y_pred_sorted)):
        y_pred_sorted.set_value(i, 'rank', (i+0.0)/len(y_pred_sorted), takeable=False)
    y_pred_fin=y_pred_sorted.sort(columns='index').reset_index(drop=True)
    y_pred_rank=y_pred_fin['rank']
    return y_pred_rank

In [ ]:
y_pred_lr_rank = rank_pred(y_predict_lr[:,1])
y_pred_gbc_rank = rank_pred(y_predict_gbc2[:,1])
y_pred_rf_rank = rank_pred(y_predict_rf[:,1])

In [ ]:
y_res_lr_rank = rank_pred(y_res_lr[:,1])
y_res_gbc_rank = rank_pred(y_res_gbc[:,1])
y_res_rf_rank = rank_pred(y_res_rf[:,1])

In [ ]:
def weightedprobressrank(w1,w2,w3):
    y_predict_ensemble =((w1/(w1+w2+w3) * y_res_lr_rank)  + (w2/(w1+w2+w3) * y_res_gbc_rank) + (w3/(w1+w2+w3) * y_res_rf_rank))
    return y_predict_ensemble

In [ ]:
y_predict_ensemble_rank = weightedprobressrank(maxi,maxj,maxk)

# Step5 - Resampling strategies


In [15]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb

under_pipe=make_pipeline_imb(RandomUnderSampler(),LogisticRegression())
scores=cross_val_score(under_pipe,X.drop(X.columns[k],axis=1),y,cv=5,scoring='roc_auc')
print scores.mean()
print cross_val_score(LogisticRegression(),X.drop(X.columns[k],axis=1),y,cv=5,scoring='roc_auc').mean()

#Improvement of 0.001 in AUC

0.791630005992
0.790642335204
